In [2]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display

In [3]:
import timeit

In [4]:
# !pip install gingerit

In [5]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i)
words_df = temp 

In [10]:
class poem_generation:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    def select_random_words(self,words_df):
        tic=timeit.default_timer()
        words_array1 = []
        while len(words_array1) <= 15:
            random_number = random.randint(0, len(words_df)-1)
            if words_df[random_number] != 0:
                words_array1.append(words_df[random_number])
        # print(len(words_array1))
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for select_random_words:",elapsed) 
        print("=========================================================")
        return words_array1

    def create_poems(self,arr):
        tic=timeit.default_timer()
        poem = [0 for _ in range(4)]
        for i in range(1,5):
            temp =  arr[4*(i-1):4*i]
            poem[i-1] = temp
            poem[i-1]=' '.join(poem[i-1])
            # print(poem[i-1])
        print("=========================================================")
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for create_poems:",elapsed) 
        return poem

    def store_syllables(self,arr):
        tic=timeit.default_timer()
        a = [0 for _ in range(4)]
        temp = 0
        j = 0
        for words,i in zip(arr,range(0,16)):
            temp = temp + syllables.estimate(words)
            if i in [3,7,11,15]:
                a[j] = temp
                # print(a[j])
                temp = 0
                j += 1
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for store_syllables:",elapsed)
        return a

    def Grammar_checker(self,text):
        tic=timeit.default_timer()
        error = 0
        poem_error = []
        tool = language_tool_python.LanguageTool('en-US')
        for lines in text:
            matches = tool.check(lines)
            if len(matches)>0:
                error = error + matches[0].errorLength
        poem_error.append(error)
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for Grammar_checker:",elapsed)
        return poem_error

    def crossover(self,p1,p2,cross):
        tic=timeit.default_timer()
        part1 = []
        part2 = []
        part3 = []
        child1 = []
        child2 = []
        j = 0
        k = 0
        random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
        random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
        for i in random_number1:
            part1.append(p1[j:i])
            j = i
        for i in random_number2:
            part2.append(p2[k:i])
            k = i
        part1.append(p1[j:])
        part2.append(p2[k:])
        part2.extend(part1)
        random.shuffle(part2)
        for i in range(len(part2)):
            part3.extend(part2[i])
        for i in range(int(len(part3)/2)):
            child1.append(part3[i])
        for i in range(int(len(part3)/2),int(len(part3))):
            child2.append(part3[i])
        if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
            child1, child2,child1_poem,child2_poem = self.crossover(child1,child2,cross)
        child1_poem = self.create_poems(child1)
        child2_poem = self.create_poems(child2)
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for crossover:",elapsed)
        return child1,child2,child1_poem,child2_poem

    def tournament(self,Population,m):
        tic=timeit.default_timer()
        p = int(m/3)
        n = m-p
        df = pd.DataFrame()
        selected_polulation = pd.DataFrame()
        Population.sort_values(by=['GrammarError'], inplace=True)
        selected_polulation = Population.iloc[:n]
        # selected_polulation = Population.loc[(Population["Syllables1"] == 5)&(Population["Syllables2"] == 6)&(Population["Syllables3"] == 6)&(Population["Syllables4"] == 5)]
        # selected_polulation = selected_polulation.iloc[:n]
        print("p:",p)
        random_number = random.sample(list(Population.iloc[n:].index),p)
        for i in random_number:
            df.append(Population.iloc[i])
        selected_polulation = pd.concat([selected_polulation,df],axis=0)
        selected_polulation = selected_polulation.reset_index(drop=True)
        print()
        print("SELECTED")
        display(selected_polulation)
        toc=timeit.default_timer()
        elapsed = toc - tic #elapsed time in seconds
        print("Time for tournament:",elapsed)
        return selected_polulation,selected_polulation["words_array"]   


    def create_set_of_poems(self,n):
            poem_population = []
            syllables_count = []
            grammar_error = []
            words_array_population = []
            for i in range(n):
                words_array = self.select_random_words(words_df)
                poem = self.create_poems(words_array)
                sum_syllables = self.store_syllables(words_array)
                poem_error = self.Grammar_checker(poem)
                poem_population.append(poem)
                syllables_count.append(sum_syllables)
                grammar_error.append(poem_error)
                words_array_population.append(words_array)
            words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
            Final = np.hstack((poem_population,syllables_count,grammar_error))
            Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
            Pop_data = pd.concat([Population_data,words_array2],axis=1)
            print()
            print("PARENTS")
            display(Pop_data)

            return Pop_data

    def child_poems(self,words_array,cross):
        child_population = []
        syllables_count = []
        grammer_error = []
        words_array_child = []
        if len(words_array)%2 == 2:
            for i in range(0,len(words_array)-1):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        else:
            for i in range(0,len(words_array)-1,2):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
        Final1 = np.hstack((child_population,syllables_count,grammer_error))
        child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
        child_data = pd.concat([child_data,words_array_child],axis=1)
        child_data = child_data.reset_index(drop=True)
        print()
        print("CHILD")
        display(child_data)
        return child_data

    def selection_from_child_parents(self,x,child_data,selected_Pop_data):
        reduce = (len(list(child_data.index)) + len(list(selected_Pop_data.index))) - x
        n = int(reduce/3)
        p = reduce - n
        if reduce > len(list(child_data.index)) + len(list(selected_Pop_data.index)):
            print("ERROR")
        else:
            if n > len(child_data):
                n = len(child_data)
                p = reduce - p
            if n == 0:
                n = 1
                p = 1
        d = pd.concat([selected_Pop_data,child_data],axis=0)
        d = d.reset_index(drop=True)
        # print("p:",p)
        # print("n:",n)
        random_number1 = random.sample(list(selected_Pop_data.index),p)
        random_number2 = random.sample(list(range(len(list(selected_Pop_data.index)),len(list(d.index)))),n)
        random_number1.extend(random_number2)
        print(random_number1)
        for i in random_number1:
          d.drop(i,axis=0,inplace = True)
        d = d.reset_index(drop=True)
        # print("NEW POPULATION")
        # display(d)
        return d
    
    def __init__(self,words_df,n,cross,epoch):
        tic_O=timeit.default_timer()
        Selection_number = int(round(0.85*n))
        assert n>1 ,f"Population {n} cannot be less than or equal to 1"
        # assert n>=Selection_number, f"Cannot select {Selection_number} poems from total population of {n}"
        assert cross>1, f"Minimum number for crossover should be 1"
        # assert 2*n >= Selection_number ,f"Cannot make initial population of {n} if only {Selection_number} parents are selected"

        Population_data = self.create_set_of_poems(n)
        for i in range(epoch):
            Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number)
            child_population = self.child_poems(selected_polulation_word_array,cross)
            final_population = self.selection_from_child_parents(n,child_population,Population_data)
            if i != epoch-1:
                print("Final Population")
                display(final_population)
                Population_data = final_population
            else:
                print("###################################################")
                print("Last Poems Standing")
                print("###################################################")
                display(final_population)
                toc_O=timeit.default_timer()
                elapsed_o = toc_O - tic_O #elapsed time in seconds
                print("Time for Total:",elapsed_o)




In [9]:
#4 (words_df,intial_population,crossover,epochs)3:
poem = poem_generation(words_df,10,2,10)

Time for select_random_words: 0.00010680000013962854
Time for create_poems: 3.749999996216502e-05
Time for store_syllables: 0.001034200000049168
Time for Grammar_checker: 28.66364829999975
Time for select_random_words: 5.520000013348181e-05
Time for create_poems: 2.3399999918183312e-05
Time for store_syllables: 0.0011073000000578759
Time for Grammar_checker: 28.86938569999984
Time for select_random_words: 6.120000034570694e-05
Time for create_poems: 0.00029429999995045364
Time for store_syllables: 0.0012058000002070912
Time for Grammar_checker: 31.955537699999695
Time for select_random_words: 5.7699999615579145e-05
Time for create_poems: 2.6599999728205148e-05
Time for store_syllables: 0.0011509999999361753
Time for Grammar_checker: 34.67857950000007
Time for select_random_words: 0.0013724999998885323
Time for create_poems: 5.419999979494605e-05
Time for store_syllables: 0.0014279000001806708
Time for Grammar_checker: 33.584701400000085
Time for select_random_words: 0.00238650000028428

,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,theres fairies as of,untoward taught universe to,Under down I in,valley forlorn again Are,6,9,4,7,20,"[theres, fairies, as, of, untoward, taught, universe, to, Under, down, I, in, valley, forlorn, again, Are]"
1,looks all falling shall,worms weeping habit sorrows,this the Away I,sad no so her,5,7,4,4,17,"[looks, all, falling, shall, worms, weeping, habit, sorrows, this, the, Away, I, sad, no, so, her]"
2,thyself Of On you,Are beats there To,me again never and,Nature sleep Have of,5,5,6,6,9,"[thyself, Of, On, you, Are, beats, there, To, me, again, never, and, Nature, sleep, Have, of]"
3,A years room men,thoroughly reclind For the,he raise not cup,veering the That his,4,7,5,5,24,"[A, years, room, men, thoroughly, reclind, For, the, he, raise, not, cup, veering, the, That, his]"
4,delight heels bat man,forgotten about nibbled fain,British come coals mead,Pythagoras Thing the are,5,8,6,8,23,"[delight, heels, bat, man, forgotten, about, nibbled, fain, British, come, coals, mead, Pythagoras, Thing, the, are]"
5,long He yesterday The,lived the our of,chase thou time wash,for down butterflies me,6,5,5,6,17,"[long, He, yesterday, The, lived, the, our, of, chase, thou, time, wash, for, down, butterflies, me]"
6,shot passage the MET,of from dew them,deid Can red garden,Hollowly cottage crescent snow,6,4,5,9,10,"[shot, passage, the, MET, of, from, dew, them, deid, Can, red, garden, Hollowly, cottage, crescent, snow]"
7,blazing You pleadedAll all,Where size And weep,it in the We,the curves rat hour,6,5,4,5,12,"[blazing, You, pleadedAll, all, Where, size, And, weep, it, in, the, We, the, curves, rat, hour]"
8,Summer set sun own,visit face They hide,unacknowledged Efface off were,above His me face,5,5,10,6,24,"[Summer, set, sun, own, visit, face, They, hide, unacknowledged, Efface, off, were, above, His, me, face]"
9,once Alter upon MY,them Is I vain,my never Farthings the,to of saw wooed,5,4,6,4,12,"[once, Alter, upon, MY, them, Is, I, vain, my, never, Farthings, the, to, of, saw, wooed]"


p: 2

SELECTED


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,shot passage the MET,of from dew them,deid Can red garden,Hollowly cottage crescent snow,6,4,5,9,10,"[shot, passage, the, MET, of, from, dew, them, deid, Can, red, garden, Hollowly, cottage, crescent, snow]"
1,blazing You pleadedAll all,Where size And weep,it in the We,the curves rat hour,6,5,4,5,12,"[blazing, You, pleadedAll, all, Where, size, And, weep, it, in, the, We, the, curves, rat, hour]"
2,once Alter upon MY,them Is I vain,my never Farthings the,to of saw wooed,5,4,6,4,12,"[once, Alter, upon, MY, them, Is, I, vain, my, never, Farthings, the, to, of, saw, wooed]"
3,looks all falling shall,worms weeping habit sorrows,this the Away I,sad no so her,5,7,4,4,17,"[looks, all, falling, shall, worms, weeping, habit, sorrows, this, the, Away, I, sad, no, so, her]"
4,long He yesterday The,lived the our of,chase thou time wash,for down butterflies me,6,5,5,6,17,"[long, He, yesterday, The, lived, the, our, of, chase, thou, time, wash, for, down, butterflies, me]"
5,theres fairies as of,untoward taught universe to,Under down I in,valley forlorn again Are,6,9,4,7,20,"[theres, fairies, as, of, untoward, taught, universe, to, Under, down, I, in, valley, forlorn, again, Are]"


Time for tournament: 0.050398500000028434
Time for create_poems: 6.620000021939632e-05
Time for create_poems: 6.759999996575061e-05
Time for crossover: 0.00043290000030538067
Time for store_syllables: 0.0023516000001109205
Time for store_syllables: 0.002551699999912671
Time for Grammar_checker: 42.49440270000014
Time for Grammar_checker: 40.1264341000001
Time for create_poems: 4.220000027999049e-05
Time for create_poems: 2.7100000352220377e-05
Time for crossover: 0.00024179999991247314
Time for store_syllables: 0.0011947999996664294
Time for store_syllables: 0.0011617000000114786
Time for Grammar_checker: 45.3397087999997
Time for Grammar_checker: 43.0735295999998
Time for create_poems: 3.1799999760551145e-05
Time for create_poems: 3.1800000215298496e-05
Time for crossover: 0.0005537000001822889
Time for store_syllables: 0.0012818999998671643
Time for store_syllables: 0.001203799999984767
Time for Grammar_checker: 42.85968550000007
Time for Grammar_checker: 45.155377900000076

CHILD


...

p: 4
n: 2
[7, 6, 8, 4, 15, 12]
NEW POPULATION


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,blazing You pleadedAll all,Where size And weep,it in the We,the curves rat hour,6,5,4,5,12,"[blazing, You, pleadedAll, all, Where, size, And, weep, it, in, the, We, the, curves, rat, hour]"
1,upon MY them Is,I vain my never,Farthings the to of,saw wooed once Alter,5,5,5,4,13,"[upon, MY, them, Is, I, vain, my, never, Farthings, the, to, of, saw, wooed, once, Alter]"
2,looks all falling shall,worms weeping habit sorrows,this the Away I,sad no so her,5,7,4,4,17,"[looks, all, falling, shall, worms, weeping, habit, sorrows, this, the, Away, I, sad, no, so, her]"
3,for fairies as of,untoward taught universe to,Under down I in,valley forlorn again Are,5,9,4,7,17,"[for, fairies, as, of, untoward, taught, universe, to, Under, down, I, in, valley, forlorn, again, Are]"
4,long down butterflies me,theres He yesterday The,lived the our of,chase thou time wash,6,7,5,5,20,"[long, down, butterflies, me, theres, He, yesterday, The, lived, the, our, of, chase, thou, time, wash]"
5,Summer set sun own,visit face They hide,unacknowledged Efface off were,above His me face,5,5,10,6,24,"[Summer, set, sun, own, visit, face, They, hide, unacknowledged, Efface, off, were, above, His, me, face]"
6,blazing You pleadedAll all,Where size And Alter,rat hour upon MY,them Is I vain,6,5,5,4,14,"[blazing, You, pleadedAll, all, Where, size, And, Alter, rat, hour, upon, MY, them, Is, I, vain]"
7,my never Farthings the,to of saw wooed,once weep it in,the We the curves,6,4,4,5,14,"[my, never, Farthings, the, to, of, saw, wooed, once, weep, it, in, the, We, the, curves]"
8,falling shall worms weeping,habit sorrows this the,Away I sad no,so her looks all,6,6,4,4,17,"[falling, shall, worms, weeping, habit, sorrows, this, the, Away, I, sad, no, so, her, looks, all]"
9,lived the our of,chase thou time wash,long down butterflies me,theres He yesterday theres,5,5,6,8,20,"[lived, the, our, of, chase, thou, time, wash, long, down, butterflies, me, theres, He, yesterday, theres]"


###################################################
Last Poems Standing
###################################################


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,blazing You pleadedAll all,Where size And weep,it in the We,the curves rat hour,6,5,4,5,12,"[blazing, You, pleadedAll, all, Where, size, And, weep, it, in, the, We, the, curves, rat, hour]"
1,upon MY them Is,I vain my never,Farthings the to of,saw wooed once Alter,5,5,5,4,13,"[upon, MY, them, Is, I, vain, my, never, Farthings, the, to, of, saw, wooed, once, Alter]"
2,looks all falling shall,worms weeping habit sorrows,this the Away I,sad no so her,5,7,4,4,17,"[looks, all, falling, shall, worms, weeping, habit, sorrows, this, the, Away, I, sad, no, so, her]"
3,for fairies as of,untoward taught universe to,Under down I in,valley forlorn again Are,5,9,4,7,17,"[for, fairies, as, of, untoward, taught, universe, to, Under, down, I, in, valley, forlorn, again, Are]"
4,long down butterflies me,theres He yesterday The,lived the our of,chase thou time wash,6,7,5,5,20,"[long, down, butterflies, me, theres, He, yesterday, The, lived, the, our, of, chase, thou, time, wash]"
5,Summer set sun own,visit face They hide,unacknowledged Efface off were,above His me face,5,5,10,6,24,"[Summer, set, sun, own, visit, face, They, hide, unacknowledged, Efface, off, were, above, His, me, face]"
6,blazing You pleadedAll all,Where size And Alter,rat hour upon MY,them Is I vain,6,5,5,4,14,"[blazing, You, pleadedAll, all, Where, size, And, Alter, rat, hour, upon, MY, them, Is, I, vain]"
7,my never Farthings the,to of saw wooed,once weep it in,the We the curves,6,4,4,5,14,"[my, never, Farthings, the, to, of, saw, wooed, once, weep, it, in, the, We, the, curves]"
8,falling shall worms weeping,habit sorrows this the,Away I sad no,so her looks all,6,6,4,4,17,"[falling, shall, worms, weeping, habit, sorrows, this, the, Away, I, sad, no, so, her, looks, all]"
9,lived the our of,chase thou time wash,long down butterflies me,theres He yesterday theres,5,5,6,8,20,"[lived, the, our, of, chase, thou, time, wash, long, down, butterflies, me, theres, He, yesterday, theres]"


Time for Total: 897.3043331999997


In [ ]:
print(list(range(5,8)))

[5, 6, 7]
